In [1]:
#!pip install PyPDF2
# directory name from which
# we are going to extract our files with its size

In [ ]:
#importing python modules
import os
import time
import glob
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import ipywidgets as widgets

#Defining get_information function which will return all the required information of the given file

def get_information(path):
        o_path=path
        
# To find the name of the file 

        x=path.rsplit("\\",1) #Spliting the path with rsplit(returns list of length two) to get the name of the file
        filename=x[1] #storing the name in the filename variable
        path=x[0]     #storing the path in the path variable
        file_info={}  #creating an empty dictionary to store all the required information of the given file
        
    
        
# To find the extension of the file 

        file_info['path'] = path #storing path in the dictionary with key as 'path' 
        file_info['filename'] = filename #storing name in the dictionary with key as 'filename'
        f_extns = filename.split(".") #splitting the filename with '.' to know the type of file
        #print ("The extension of the file is : " + repr(f_extns[-1]))
        file_info['file_extension']= f_extns[-1]#storing the fileextension in the fileinfo dictionary with 'file_extension'askey
        
# To find the hash value of the file 
    
        if os.path.isfile(o_path):
            import hashlib
            f = open(o_path,'rb')
            x = f.read() #x will have the type 'bytes'
            md5_value = hashlib.md5(x).hexdigest()
            file_info['hash_value']=md5_value
            f.close()
        else:
            file_info['hash_value']=None
            
# To find the size of file
        
        size=os.path.getsize(o_path)#using the getsize method from the os module to get the size of file 
        file_info['size']=size
        
# To find the last date of modification 

        date_of_modification = os.path.getmtime(path) #using the getmtime function to get the last time the file is modified
        date_of_modification = time.ctime(date_of_modification) #Changing the time in seconds to time stamp with ctime method
        file_info['date_of_modification']= date_of_modification #storing the date in the file_info dictionary
        
        return file_info #returning the whole file_info dictionary 
    
    
        
        
    


In [ ]:
''' This is the core part of the project, where the class file_pro will take the path of the directory of the folder. The constr
uctor will checks wheter the excel file with name file_info present and if present we will check last date of modification and
run the get_information for the entire folder if the date modified is greater than two days.The get_info method in class will 
save the fileinfo information into the excel file with name fileinfo.generate_report method will gives the report.'''

class file_pro:
    def __init__(self,path): #constructor 
        self.path=path
        if os.path.exists('fileinfo.xlsx'): #checking the fileinfo file is present in the given directory
            file=pd.read_excel('fileinfo.xlsx') #reading the excel file using the pandas data frame
            #print('1',  file)
            ctime=file.loc[0]['date_of_modification'] #getting the last date of the modification of the entire file
            stime=datetime.strptime(ctime, '%a %b %d %H:%M:%S %Y')#converting the ctime into datetime object
            now=datetime.now()#getting the present time using now function in datetime module
            now=pd.date_range(now,periods=1,freq='S')#converting the time into pandas timeframe to check the difference
            before=pd.date_range(stime,periods=1,freq='S')#converting the last modified time into pandas timeframe
            days=now-before#subtracting the present time and the time last modified
            days=days/np.timedelta64(1,'D')#getting the timedifference in days
            days=str(days[0]).split(' ')#eliminating the hours part of the difference
            days=float(days[0]) 
            if days>=2:#checking wether the last date modified was greater than 2 days  
                file={}#creating the dictionary 
                file[0]=get_information(path)
                path=self.path+'/**'
                index1=1
                for path in glob.glob(path ,recursive=True):
                    path.encode('unicode_escape')
                    if os.path.isfile(path):
                        file[index1]=get_information(path)
                        index1+=1
                self.file=file
                df=pd.DataFrame(file)
                df=df.T
                self.file=df
            else:
                self.file=file
                #print(file)
        else:
            file={}
            file[0]=get_information(path)
            path=self.path+'/**'
            index1=1
            for path in glob.glob(path ,recursive=True):
                path.encode('unicode_escape')
                if os.path.isfile(path):
                    file[index1]=get_information(path)
                    index1+=1
            self.file=file
            df=pd.DataFrame(file)
            df=df.T
            self.file=df
            
        
    def get_info(self):
        file = self.file
        file.to_excel('fileinfo.xlsx',index=False)
# This is my original first functionality
        df=file
        ALL='ALL'
        def unique_sorted_values_plus_ALL(array):
            unique = array.unique().tolist()
            unique.sort()
            unique.insert(0, ALL)
            return unique
        filename = widgets.Dropdown(options =    unique_sorted_values_plus_ALL(df.filename))
        def file_name_handler(change):
            if (change.new == ALL):
                display(df)
            else:
                display(df[df.filename == change.new])
        filename.observe(file_name_handler,names='value')
        display(filename)
        return file
        
        
    def generate_report(self):
        
# To get the extensions of all the files that are present in the given directory

        file=self.file
        path=self.path
        extensions=pd.unique(file['file_extension'])
        l=list(extensions)
        print(extensions)
        print('_________________________')
        
        
        for ext in extensions:
            df=self.file
            print("The mean and median size of all "+ ext +" files " + str(np.mean(df[df['file_extension']==ext]['size']))+' '+
                  str(np.median(df[df['file_extension']==ext]['size'])))
            l.append("The mean and median size of all "+ ext +" files " + str(np.mean(df[df['file_extension']==ext]['size']))+' '+
                  str(np.median(df[df['file_extension']==ext]['size'])))
        
        
        print('_________________________')
        
        folders={}
        path=path+'/*'
        num=0
        for item in glob.glob(path,recursive=True):
            folder={}
            m=[]
            if os.path.isdir(item):
                #print(item)
                folder_name=item.rsplit("\\" ,1 )[1]
                folder['folder_name']=folder_name
                item=item+'/**'
                count=0
                size=0
                for file in glob.glob(item,recursive=True):
                    if os.path.isfile(file):
                        #print(file)
                        count+=1
                        size+=get_information(file)['size']
                folder['count_of_files']=count
                folder['size_of_files']=size
                m.append('folder_name '+ folder_name)
                m.append('count of files ' + str(count))
                m.append('size of file '+ str(size))
                folders[num]=folder
                num+=1
                l.append(m)
        folders=pd.DataFrame(folders).T
        print(folders)
        #Below bargraphs are the second original functionality
        sns.barplot(x='folder_name',y='count_of_files',data=folders)
        plt.show()
        sns.barplot(x='folder_name',y='size_of_files',data = folders)
        plt.show()
        Decision=input('Do you want to save this report ')
        if Decision=='Yes' or 'yes':
            l=np.array(l,dtype='object')
            file = open("file1.txt", "w+")
            content = str(l)
            file.write(content)
            file.close()
        
        
                
    
    
    
    
        
    
            
        
        
        
        
        
        
        
        
    

In [ ]:
path=os.getcwd()
u=file_pro(path)
#get_information(path)


In [ ]:
u.generate_report()

In [ ]:
path=os.getcwd()
print(get_information(path))


In [ ]:
u.get_info()